In [1]:
# pytorch
import datetime
from pandas.core.tools.datetimes import to_datetime
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
import torch.optim as optim


import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from statsmodels.tsa.seasonal import seasonal_decompose
import numpy as np
# third - party
import pandas as pd 
from api.dataset import DatasetConstructur
# python imports
import random
import requests
import json
from datetime import date, timedelta
from api.rnn import RNN


In [2]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


STOCK_LIST = ['AAPL', 'AXP', 'KO', 'WMT', 'DIS', 'IBM', 'NKE', 'V']


data = DatasetConstructur(
    STOCK_LIST, date(2020, 1, 1), date(2021, 1, 1),15)


Getting:  AAPL  from  2020-01-01  to  2021-01-01
Getting:  AXP  from  2020-01-01  to  2021-01-01
Getting:  KO  from  2020-01-01  to  2021-01-01
Getting:  WMT  from  2020-01-01  to  2021-01-01
Getting:  DIS  from  2020-01-01  to  2021-01-01
Getting:  IBM  from  2020-01-01  to  2021-01-01
Getting:  NKE  from  2020-01-01  to  2021-01-01
Getting:  V  from  2020-01-01  to  2021-01-01


In [3]:
x = random.sample(data.datasets, len(data.datasets))


In [4]:
training_datasets = x[:84]
testing_datasets = x[84:120]

In [ ]:
len(training_datasets)

In [5]:
training_tensor =  torch.empty(14, 15, data.input_size)
training_labels = torch.empty(14)

training_batches = []
total_size = 0
for x in range(6):
    for n in range(14):

        datagroup = training_datasets[total_size]
        training_labels[n] = int(datagroup['label'])
    
        df = datagroup['dataset']

        x = 0
        for _, row in df.iterrows():
            y = 0
            for col in row:

                training_tensor[n][x][y] = col
                y = y + 1
            x = x + 1
        
        total_size = total_size + 1
    training_batches.append({
        'training_tensor': training_tensor,
        'labels': training_labels
    })

    



In [6]:
num_layers = 3
hidden_size = 2
sequence_length = 15
model = RNN(data.input_size, 104, 2, data.input_size, sequence_length)

In [7]:
EPOCHS = 2
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(EPOCHS):
    for data in training_batches:

        X_tensor = data['training_tensor']
        labels = data['labels']
        model.zero_grad()
        print(labels)
        output = model(X_tensor.view(-1, 15,35))
        print(output)
        loss = F.nll_loss(output, labels)
        print(loss)



tensor([1., 0., 0., 1., 1., 0., 0., 0., 1., 1., 0., 1., 0., 0.])


TypeError: cat() received an invalid combination of arguments - got (Tensor, dim=int), but expected one of:
 * (tuple of Tensors tensors, int dim, *, Tensor out)
 * (tuple of Tensors tensors, name dim, *, Tensor out)


In [ ]:
l1 = [1,2,3,4,5,6,7]